### Association rules by Department Bakery
#### Department - Alchohol

In [1]:
##importing libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import OnehotTransactions
import pandas as pd
from sklearn import cluster, datasets
import numpy as np
import scipy
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

In [2]:
# Csv contains data of trainorders and prior orders
combine_train_prior=pd.read_csv("CombineOrders1.csv",encoding = "ISO-8859-1")

In [3]:
## altering datatypes for ease of use
combine_train_prior['aisle_id']=combine_train_prior['aisle_id'].astype('int16')
combine_train_prior['order_id']=combine_train_prior['order_id'].astype('int32')
combine_train_prior['product_id']=combine_train_prior['product_id'].astype('int32')
combine_train_prior['add_to_cart_order']=combine_train_prior['add_to_cart_order'].astype('int16')
combine_train_prior['reordered']=combine_train_prior['reordered'].astype('int8')
combine_train_prior['department_id']=combine_train_prior['department_id'].astype('int8')



In [4]:
combine_train_prior['product_name'] = combine_train_prior['product_name'].str.strip()
combine_train_prior.dropna(axis=0, subset=['order_id'], inplace=True)

In [5]:
## this will give us the popular departments as departments with the highest value counts will be those from where consumers have purchased the most
##Top 5 Departments are produce,dairy eggs, snacks, beverages, frozen
combine_train_prior['department'].value_counts()

produce            9888378
dairy eggs         5631067
snacks             3006412
beverages          2804175
frozen             2336858
pantry             1956819
bakery             1225181
canned goods       1114857
deli               1095540
dry goods pasta     905340
household           774652
meat seafood        739238
breakfast           739069
personal care       468693
babies              438743
international       281155
alcohol             159294
pets                102221
missing              77396
other                38086
bulk                 35932
Name: department, dtype: int64

In [6]:
combine_train_prior

,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,0,1,49302,1,1,112108,train,4,4,10,9.0,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,1,1,43633,5,1,112108,train,4,4,10,9.0,Lightly Smoked Sardines in Olive Oil,95,15,canned meat seafood,canned goods
2,2,1,10246,3,0,112108,train,4,4,10,9.0,Organic Celery Hearts,83,4,fresh vegetables,produce
3,3,1,22035,8,1,112108,train,4,4,10,9.0,Organic Whole String Cheese,21,16,packaged cheese,dairy eggs
4,4,1,47209,7,0,112108,train,4,4,10,9.0,Organic Hass Avocado,24,4,fresh fruits,produce
5,5,1,13176,6,0,112108,train,4,4,10,9.0,Bag of Organic Bananas,24,4,fresh fruits,produce
6,6,1,11109,2,1,112108,train,4,4,10,9.0,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,other creams cheeses,dairy eggs
7,7,1,49683,4,0,112108,train,4,4,10,9.0,Cucumber Kirby,83,4,fresh vegetables,produce
8,8,36,46979,8,1,79431,train,23,6,18,30.0,Asparagus,83,4,fresh vegetables,produce
9,9,36,39612,1,0,79431,train,23,6,18,30.0,Grated Pecorino Romano Cheese,2,16,specialty cheeses,dairy eggs


In [7]:
## Using Apriori we will find if there are association rules found for products bought from department Dairy Eggs
dept_produce=combine_train_prior[combine_train_prior['department'] =="alcohol"].reset_index()

In [8]:
## Preparing dataset for Apriori
temp_df=dept_produce.loc[:,['order_id','product_name']]

In [9]:
## In order to find association we have considered those orders that have products which 4 or more in number
grouped_data=temp_df.groupby("order_id").filter(lambda x: len(x) >=4)

In [10]:
grouped_data

,order_id,product_name
7,5466,Pinot Noir
8,5466,Chianti Classico
9,5466,Chardonnay Sonoma Coast 2008
10,5466,Imported Beer
11,5466,Icelandic White Ale 6 Pack
45,37345,Whiskey
46,37345,Frontier Whiskey
47,37345,"Handmade Vodka From Austin, Texas"
48,37345,London Dry Gin
50,40123,Vodka


In [11]:
## Pivoting the table to get occurences of a product in a particuler order
from pandas import *
# table=pd.DataFrame()
table1=pivot_table(grouped_data,index='order_id',columns='product_name',aggfunc=len)

In [12]:
#relace NAN with 0
table1=table1.fillna(0)

In [13]:
## of the product exists then value changed to 1 else 0
def encode_units(x):
    if x == 0.0:
        return 0
    if x >= 1.0:
        return 1


In [14]:
basket_sets = table1.applymap(encode_units)

In [15]:
## Running Apriori to get frequent itemset with a minimum support of 0.01. ie the combination must exist atleast in 1% of dataset
frequent_itemsets = apriori(table1, min_support=0.01, use_colnames=True)

In [16]:
frequent_itemsets

,support,itemsets
0,0.012896,[12 Oz Beer]
1,0.013021,[12 Oz Lager]
2,0.012395,[312 Urban Wheat Ale]
3,0.011644,[80 Vodka Holiday Edition]
4,0.022537,[805]
5,0.029047,"[Ale, India Pale, Brew Free! Or Die IPA]"
6,0.076249,[Amber Ale]
7,0.232753,[Beer]
8,0.012395,[Beer Cans]
9,0.022662,[Belgian Style Wheat Ale]


In [17]:
# Getting the rules of association based on the minimum support selected
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [18]:
rules

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(India Pale Ale),(805),0.210592,0.022537,0.011519,0.054697,2.427018,0.006773,1.034021
1,(805),(India Pale Ale),0.022537,0.210592,0.011519,0.511111,2.427018,0.006773,1.614698
2,(Beer),(Longboard Island Lager),0.232753,0.039815,0.018029,0.077461,1.945538,0.008762,1.040807
3,(Longboard Island Lager),(Beer),0.039815,0.232753,0.018029,0.452830,1.945538,0.008762,1.402210
4,(Pinot Noir Wine),(Chardonnay),0.054839,0.183423,0.014273,0.260274,1.418982,0.004214,1.103891
5,(Chardonnay),(Pinot Noir Wine),0.183423,0.054839,0.014273,0.077816,1.418982,0.004214,1.024915
6,(Red Blend),(Sauvignon Blanc),0.075498,0.214724,0.021911,0.290216,1.351575,0.005699,1.106359
7,(Sauvignon Blanc),(Red Blend),0.214724,0.075498,0.021911,0.102041,1.351575,0.005699,1.029559
8,(Beer),(Belgium Beer),0.232753,0.043195,0.016777,0.072082,1.668745,0.006723,1.031131
9,(Belgium Beer),(Beer),0.043195,0.232753,0.016777,0.388406,1.668745,0.006723,1.254503


In [19]:
## Since, ideally Lift must greater than 1 , we have selected the rules to have Lift to be greater than 1, confidence greater than 30% and a conviction of 40%
final_result=rules[ (rules['lift'] > 1) &
       (rules['confidence'] >= 0.3) & rules['conviction']<1.4 ]
final_result

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(India Pale Ale),(805),0.210592,0.022537,0.011519,0.054697,2.427018,0.006773,1.034021
1,(805),(India Pale Ale),0.022537,0.210592,0.011519,0.511111,2.427018,0.006773,1.614698
2,(Beer),(Longboard Island Lager),0.232753,0.039815,0.018029,0.077461,1.945538,0.008762,1.040807
3,(Longboard Island Lager),(Beer),0.039815,0.232753,0.018029,0.452830,1.945538,0.008762,1.402210
4,(Pinot Noir Wine),(Chardonnay),0.054839,0.183423,0.014273,0.260274,1.418982,0.004214,1.103891
5,(Chardonnay),(Pinot Noir Wine),0.183423,0.054839,0.014273,0.077816,1.418982,0.004214,1.024915
6,(Red Blend),(Sauvignon Blanc),0.075498,0.214724,0.021911,0.290216,1.351575,0.005699,1.106359
7,(Sauvignon Blanc),(Red Blend),0.214724,0.075498,0.021911,0.102041,1.351575,0.005699,1.029559
8,(Beer),(Belgium Beer),0.232753,0.043195,0.016777,0.072082,1.668745,0.006723,1.031131
9,(Belgium Beer),(Beer),0.043195,0.232753,0.016777,0.388406,1.668745,0.006723,1.254503


In [20]:
final_result.to_csv("Department_alchoholApriori.csv",sep=',')

In [21]:
test1=pd.read_csv("Department_alchoholApriori.csv")

In [22]:
## Cleaning the resultset for usability
test1=test1.replace({'frozenset': ''}, regex=True)

In [23]:
test1=test1.replace({"u'": ''}, regex=True)

In [24]:
test1['antecedants'] = test1['antecedants'].str.strip('()').str.strip('[]')
test1['antecedants'] = test1['antecedants'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['antecedants'] = test1['antecedants'].str.replace("'",'')

In [25]:
test1['consequents'] = test1['consequents'].str.strip('()')
test1['consequents'] = test1['consequents'].str.strip('[]')
test1['consequents'] = test1['consequents'].str.strip("'")
test1['consequents'] = test1['consequents'].str.replace("'",'')

In [26]:
test1=test1.drop(['Unnamed: 0'], axis = 1)

In [27]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,India Pale Ale,805,0.210592,0.022537,0.011519,0.054697,2.427018,0.006773,1.034021
1,805,India Pale Ale,0.022537,0.210592,0.011519,0.511111,2.427018,0.006773,1.614698
2,Beer,Longboard Island Lager,0.232753,0.039815,0.018029,0.077461,1.945538,0.008762,1.040807
3,Longboard Island Lager,Beer,0.039815,0.232753,0.018029,0.452830,1.945538,0.008762,1.402210
4,Pinot Noir Wine,Chardonnay,0.054839,0.183423,0.014273,0.260274,1.418982,0.004214,1.103891
5,Chardonnay,Pinot Noir Wine,0.183423,0.054839,0.014273,0.077816,1.418982,0.004214,1.024915
6,Red Blend,Sauvignon Blanc,0.075498,0.214724,0.021911,0.290216,1.351575,0.005699,1.106359
7,Sauvignon Blanc,Red Blend,0.214724,0.075498,0.021911,0.102041,1.351575,0.005699,1.029559
8,Beer,Belgium Beer,0.232753,0.043195,0.016777,0.072082,1.668745,0.006723,1.031131
9,Belgium Beer,Beer,0.043195,0.232753,0.016777,0.388406,1.668745,0.006723,1.254503


In [28]:
## Writing results to a csv file
test1.to_csv("FinalDepartment_alchoholApriori.csv")

In [29]:
test1

,antecedants,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,India Pale Ale,805,0.210592,0.022537,0.011519,0.054697,2.427018,0.006773,1.034021
1,805,India Pale Ale,0.022537,0.210592,0.011519,0.511111,2.427018,0.006773,1.614698
2,Beer,Longboard Island Lager,0.232753,0.039815,0.018029,0.077461,1.945538,0.008762,1.040807
3,Longboard Island Lager,Beer,0.039815,0.232753,0.018029,0.452830,1.945538,0.008762,1.402210
4,Pinot Noir Wine,Chardonnay,0.054839,0.183423,0.014273,0.260274,1.418982,0.004214,1.103891
5,Chardonnay,Pinot Noir Wine,0.183423,0.054839,0.014273,0.077816,1.418982,0.004214,1.024915
6,Red Blend,Sauvignon Blanc,0.075498,0.214724,0.021911,0.290216,1.351575,0.005699,1.106359
7,Sauvignon Blanc,Red Blend,0.214724,0.075498,0.021911,0.102041,1.351575,0.005699,1.029559
8,Beer,Belgium Beer,0.232753,0.043195,0.016777,0.072082,1.668745,0.006723,1.031131
9,Belgium Beer,Beer,0.043195,0.232753,0.016777,0.388406,1.668745,0.006723,1.254503


In [30]:
## testing the result set
t=test1[test1['antecedants'] =="Organic Strawberries, Organic Baby Spinach"].reset_index()